In [2]:
import gradio as gr
from transformers import pipeline
import numpy as np

In [3]:
import torch
device = "cuda:0" if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [4]:
# model = "facebook/hubert-large-ls960-ft" # for all languages
model = "openai/whisper-base.en" # for english
# model = "ai4bharat/indicwav2vec-hindi" # for hindi, gujarati
# model = "facebook/wav2vec2-large-960h-lv60-self" # for deu, eng, fra, pol

# TODO: implement language adapters load_adapter() function  
# model = "facebook/mms-1b-fl102" # ara, cmn-script_simplified, deu, eng, fra, guj, hin, jpn, pol, tam, tgl, urd-script_arabic, yue-script_traditional



In [6]:
language = 'eng' # 'hin'
# transcriber = pipeline("automatic-speech-recognition", model=model,model_kwargs={"target_lang": language, "ignore_mismatched_sizes": True}, device=device)
transcriber = pipeline("automatic-speech-recognition", model=model, device=device)

generation_config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [7]:
import torchaudio
from transformers import  AutoProcessor

waveform, sample_rate = torchaudio.load("audios/male.wav", format="wav")
print(sample_rate)

processor = AutoProcessor.from_pretrained(model)
inputs = processor(waveform[0], sampling_rate=16000, return_tensors="pt")


8000


In [19]:
inputs['input_features'].numpy()[0][0]

array([ 0.00222659, -0.1440072 , -0.1776303 , ..., -0.6423832 ,
       -0.6423832 , -0.6423832 ], dtype=float32)

In [21]:
with torch.no_grad():
    outputs = transcriber(inputs['input_features'].numpy()[0][0])


In [22]:
outputs

{'text': ' you'}

In [ ]:
def transcribe_with_stream(stream, new_chunk):
    sr, y = new_chunk
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    if stream is not None:
        stream = np.concatenate([stream, y])
    else:
        stream = y

    transcribed = transcriber({"sampling_rate": sr, "raw": stream})
    print(transcribed)
    text = transcribed["text"]
    return stream, text

demo = gr.Interface(
    transcribe_with_stream,
    ["state", gr.Audio(sources=["microphone"], streaming=True)],
    ["state", "text"],
    live=True,
)

demo.launch()
